In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.optimizers import Adam

def load_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img = os.path.join(folder_path, filename)
        images.append(img)
        labels.append(label)
    return images, labels

# Define the folders containing the images and their corresponding labels
folder_path_classA = r"D:\terrian-classification-main\train\Grassy"
folder_path_classB = r"D:\terrian-classification-main\train\Marshy"
folder_path_classC = r"D:\terrian-classification-main\train\Rocky"
folder_path_classD = r"D:\terrian-classification-main\train\Sandy"

# Load images and labels for class A
images_classA, labels_classA = load_images_from_folder(folder_path_classA, label="Grassy")

# Load images and labels for class B
images_classB, labels_classB = load_images_from_folder(folder_path_classB, label="Marshy")

# Load images and labels for class A
images_classC, labels_classC = load_images_from_folder(folder_path_classA, label="Rocky")

# Load images and labels for class B
images_classD, labels_classD = load_images_from_folder(folder_path_classB, label="Sandy")

# Combine the data from both classes into a single DataFrame
data = {
    "image": images_classA + images_classB+images_classC + images_classD,
    "label": labels_classA + labels_classB+images_classC + images_classD
}

df = pd.DataFrame(data)

# Shuffle the DataFrame (optional)
# df = df.sample(frac=1).reset_index(drop=True)

# Encode labels
label_encoder = LabelEncoder()
df["encoded_label"] = label_encoder.fit_transform(df["label"])

# Load and preprocess images
def load_and_preprocess_image(image_path, image_width, image_height):
    image = Image.open(image_path)
    image = image.resize((image_width, image_height))
    image_array = np.array(image)
    
    # If the image is grayscale, convert it to RGB
    if len(image_array.shape) == 2:
        image_array = np.stack((image_array,) * 3, axis=-1)
    
    image_array = image_array / 255.0  # Normalize pixel values
    
    return image_array

# Set the target dimensions for resizing the images
target_width = 224
target_height = 224

X = np.array([load_and_preprocess_image(str(path), target_width, target_height) for path in df['image']])
y = to_categorical(df['encoded_label'])

# Load the model from the H5 file
loaded_model = load_model('densenet121_model.h5')

# Print the shape of the feature extraction layer for debugging
print("Feature Shape:", loaded_model.layers[-3].output_shape[1:])

# Define your transformer model
def transformer_model(input_shape, num_transformer_layers, output_dim):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Flatten the variable dimensions and keep the channel dimension intact
    flattened_inputs = layers.Flatten()(inputs)
    
    # Transformer layers using MultiHeadAttention
    sequence = MultiHeadAttention(num_heads=5, key_dim=64)(flattened_inputs, flattened_inputs)
    sequence = layers.LayerNormalization(epsilon=1e-6)(sequence)
    sequence = layers.Dropout(0.1)(sequence)
    
    # Reshape the sequence to maintain 3D structure
    sequence = layers.Reshape((-1, 1024))(sequence)  # Assuming the channel dimension is 1024
    
    # Output layer for binary classification with sigmoid activation
    outputs = layers.Dense(output_dim, activation='sigmoid')(sequence)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Define your task-specific output dimension (binary classification)
output_dim = 2  # Two classes: "Class A" and "Class B"
num_tlayers = 2

# Create the transformer model
transformer = transformer_model(input_shape=(64, 64, 1024), num_transformer_layers=num_tlayers, output_dim=output_dim)

# Print the summary of the model for debugging
transformer.summary()

# Combine the pre-trained feature extractor with the transformer
combined_model = models.Sequential([
    loaded_model,
    transformer,
    layers.Flatten()  # Add Flatten layer to reshape the output
])

# Compile the model (adjust loss and metrics based on your task)
combined_model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the combined model
combined_model.summary()


Feature Shape: (64,)


ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [11]:

# Train the combined model on your task-specific data
combined_model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
27/27 [==============================] - 42s 550ms/step - loss: 0.9858 - accuracy: 0.6064 - val_loss: 0.3629 - val_accuracy: 0.9009
Epoch 2/20
27/27 [==============================] - 6s 239ms/step - loss: 0.2212 - accuracy: 0.9409 - val_loss: 0.1643 - val_accuracy: 0.9575
Epoch 3/20
27/27 [==============================] - 6s 238ms/step - loss: 0.1302 - accuracy: 0.9681 - val_loss: 0.1161 - val_accuracy: 0.9575
Epoch 4/20
27/27 [==============================] - 6s 237ms/step - loss: 0.0893 - accuracy: 0.9775 - val_loss: 0.1038 - val_accuracy: 0.9575
Epoch 5/20
27/27 [==============================] - 7s 252ms/step - loss: 0.0672 - accuracy: 0.9835 - val_loss: 0.0917 - val_accuracy: 0.9717
Epoch 6/20
27/27 [==============================] - 7s 253ms/step - loss: 0.0469 - accuracy: 0.9894 - val_loss: 0.0840 - val_accuracy: 0.9764
Epoch 7/20
27/27 [==============================] - 7s 253ms/step - loss: 0.0315 - accuracy: 0.9941 - val_loss: 0.0798 - val_accuracy: 0.9764
Epoch

In [12]:

# Train the combined model on your task-specific data
combined_model.fit(X,y, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
27/27 [==============================] - 7s 259ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0869 - val_accuracy: 0.9811
Epoch 2/10
27/27 [==============================] - 7s 252ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.0871 - val_accuracy: 0.9811
Epoch 3/10
27/27 [==============================] - 7s 272ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0880 - val_accuracy: 0.9811
Epoch 4/10
27/27 [==============================] - 7s 275ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.0899 - val_accuracy: 0.9811
Epoch 5/10
27/27 [==============================] - 7s 261ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0913 - val_accuracy: 0.9811
Epoch 6/10
27/27 [==============================] - 7s 256ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0924 - val_accuracy: 0.9811
Epoch 7/10
27/27 [==============================] - 7s 259ms/step - loss: 0.0079 - accuracy: 0.9965 - val_loss: 0.0890 - val_accuracy: 0.9764
Epoch 

In [13]:
import tensorflow as tf

# Check if TensorFlow is using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Print GPU details
print("GPU Details:")
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    print(gpu)


Num GPUs Available:  1
GPU Details:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [16]:
combined_model.save('VIT.plt')

INFO:tensorflow:Assets written to: VIT.plt\assets


INFO:tensorflow:Assets written to: VIT.plt\assets


In [1]:
import os
import pandas as pd
import numpy as np

def load_images_from_folder(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img = os.path.join(folder_path, filename)
        # if img is not None:
        #     img = cv2.resize(img, (227, 227))  # Resize the images to match the input shape of AlexNet
        #     img=np.array(img)
        images.append(img)
        labels.append(label)
    return images, labels

# Define the two folders containing the images and their corresponding labels
folder_path_classA = r"D:\suresh\data4-tuned\test\RAW"
folder_path_classB = r"D:\suresh\data4-tuned\test\partially_ripe"
folder_path_classC = r"D:\suresh\data4-tuned\test\medium_ripe"
folder_path_classD = r"D:\suresh\data4-tuned\test\fully_ripe"
# Load images and labels for classA
images_classA, labels_classA = load_images_from_folder(folder_path_classA, label="Raw")

# Load images and labels for classB
images_classB, labels_classB = load_images_from_folder(folder_path_classB, label="Partial")

images_classC, labels_classC = load_images_from_folder(folder_path_classC, label="Medium")

# Load images and labels for classD
images_classD, labels_classD = load_images_from_folder(folder_path_classD, label="Full")

# Combine the data from both classes into a single DataFrame
data = {
    "image": images_classA + images_classB+images_classC + images_classD,
    "label": labels_classA + labels_classB+labels_classC + labels_classD
}

dft = pd.DataFrame(data)

# Shuffle the DataFrame (optional)
dft = dft.sample(frac=1).reset_index(drop=True)

# Now, the DataFrame 'df' contains the image data and corresponding labels from both folders.
# You can use this DataFrame for further processing or training your model.


In [2]:
from sklearn.preprocessing import LabelEncoder
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels in the DataFrame
dft["encoded_label"] = label_encoder.fit_transform(dft["label"])

print(dft)


                                                 image    label  encoded_label
0    D:\suresh\data4-tuned\test\medium_ripe\ripe834...   Medium              1
1    D:\suresh\data4-tuned\test\partially_ripe\ripe...  Partial              2
2             D:\suresh\data4-tuned\test\RAW\raw87.jpg      Raw              3
3    D:\suresh\data4-tuned\test\medium_ripe\ripe729...   Medium              1
4            D:\suresh\data4-tuned\test\RAW\raw294.jpg      Raw              3
..                                                 ...      ...            ...
183  D:\suresh\data4-tuned\test\medium_ripe\ripe326...   Medium              1
184  D:\suresh\data4-tuned\test\partially_ripe\ripe...  Partial              2
185           D:\suresh\data4-tuned\test\RAW\raw17.jpg      Raw              3
186  D:\suresh\data4-tuned\test\partially_ripe\ripe...  Partial              2
187  D:\suresh\data4-tuned\test\medium_ripe\ripe295...   Medium              1

[188 rows x 3 columns]


In [3]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
def load_and_preprocess_image(image_path, image_width, image_height):
    image = Image.open(image_path)
    image = image.resize((image_width, image_height))
    image_array = np.array(image) / 255.0
    return image_array

In [5]:
from tensorflow.keras.utils import to_categorical
Xt = np.array([load_and_preprocess_image(str(path), 224,224) for path in dft['image']])
yt = to_categorical(dft['encoded_label']) 

In [6]:
from tensorflow.keras.models import load_model

# Load the model from the H5 file
model = load_model('VIT.plt')

# Now, you can use the loaded_model to make predictions on new data
# For example:
import numpy as np

# Assuming you have new data stored in the 'new_data' variable
predictions = model.predict(Xt)

# 'predictions' will contain the model's output for the new data
# You can now use 'predictions' for further processing or analysis


In [7]:
thresholded_predictions = np.zeros_like(predictions)
thresholded_predictions[np.arange(len(predictions)), predictions.argmax(axis=1)] = 1

print(thresholded_predictions)

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0.

In [8]:
from sklearn.metrics import classification_report

In [9]:
# Create a classification report
report = classification_report(yt,thresholded_predictions)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.80      0.94      0.86        34
           1       0.87      0.78      0.82        60
           2       0.65      0.56      0.60        27
           3       0.87      0.93      0.90        67

   micro avg       0.83      0.83      0.83       188
   macro avg       0.80      0.80      0.80       188
weighted avg       0.83      0.83      0.83       188
 samples avg       0.83      0.83      0.83       188



In [10]:
from sklearn.metrics import multilabel_confusion_matrix
# Create a classification report
report1 = multilabel_confusion_matrix(yt, thresholded_predictions)

# Print the classification report
print(report1)

[[[146   8]
  [  2  32]]

 [[121   7]
  [ 13  47]]

 [[153   8]
  [ 12  15]]

 [[112   9]
  [  5  62]]]


In [11]:
# Calculate class-wise accuracy
class_accuracy = np.array([np.diag(conf_matrix) / np.sum(conf_matrix, axis=1) for conf_matrix in report1])

# Convert to percentage
class_accuracy_percentage = class_accuracy * 100

# Display class-wise accuracy
for i, acc in enumerate(class_accuracy_percentage):
    print(f"Class {i}: {acc[1]:.2f}%")  # Assuming the positive class is in the second row (index 1)

Class 0: 94.12%
Class 1: 78.33%
Class 2: 55.56%
Class 3: 92.54%


In [12]:
# Extract true positives (TP) from the confusion matrix
TP = report1[:, 1, 1]  # Assuming positive class is 1, modify accordingly

# Calculate accuracy
accuracy = TP.sum() / len(yt)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 82.98%
